In [16]:
from llama_index.core import (
    ServiceContext,
    PromptHelper,
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SimpleNodeParser
import tiktoken
# from ..prompts import RecommendationPrompt
from typing import List
import logging
import dotenv
import os

In [17]:
dotenv.load_dotenv()
# read OPENAI_API_KEY from .env file
api_key = os.getenv("OPENAI_API_KEY")

In [18]:
from pydantic import BaseModel

class RecommendationPrompt(BaseModel):

    template: str = """
        {details}
    """

    details: str

    def __str__(self):

        return self.template.format(details = self.details)

In [19]:
class RAGProvider:

    def __init__(
        self,
        api_key,
        model: str = 'gpt-3.5-turbo',
        persist_dir: str = "tmp"
    ):

        self.model = model
        self.persist_dir = persist_dir

        self.llm = OpenAI(
            model = model,
            temperature = 0,
            max_tokens = 256,
            api_key = api_key
        )

        self.embed_model = OpenAIEmbedding(api_key = api_key)

    def create_index(
        self,
        separator: str = " ",
        chunk_size: int = 1024,
        chunk_overlap: int = 20,
        prompt_context_window: int = 4096,
        num_output: int = 256,
        chunk_overlap_ratio: float = 0.1,
        chunk_size_limit = None,
        input_files: str = "butil/tmp/"
    ):

        node_parser = SimpleNodeParser.from_defaults(
            separator = separator,
            chunk_size = chunk_size,
            chunk_overlap = chunk_overlap,
            tokenizer = tiktoken.encoding_for_model(self.model).encode
        )

        prompt_helper = PromptHelper(
            context_window = prompt_context_window, 
            num_output = num_output, 
            chunk_overlap_ratio = chunk_overlap_ratio, 
            chunk_size_limit = chunk_size_limit
        )

        service_context = ServiceContext.from_defaults(
            llm = self.llm,
            embed_model = self.embed_model,
            node_parser = node_parser,
            prompt_helper = prompt_helper
        )

        documents = SimpleDirectoryReader(input_files).load_data()

        index = VectorStoreIndex.from_documents(
            documents, 
            service_context = service_context
        )

        index.storage_context.persist(persist_dir = self.persist_dir)

    def query(
        self,
        prompt: RecommendationPrompt,
        index_dir: str = "tmp/"
    ) -> str:
        
        storage_context = StorageContext.from_defaults(persist_dir = index_dir)
        index = load_index_from_storage(storage_context = storage_context)
        query_engine = index.as_query_engine()

        formatted_prompt = str(prompt)
        logging.info(f"Formatted prompt: {formatted_prompt}")
        response = query_engine.query(formatted_prompt)
        return response

In [20]:
provider = RAGProvider(api_key = api_key)

In [21]:
# Check if index already exists
if not os.path.exists(os.path.join(provider.persist_dir)):
    provider.create_index()

/var/folders/0j/d0xd373d2tb7zdzcsqrk3mpw0000gn/T/ipykernel_19558/4264067201.py:48: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [11]:
prompt_question = RecommendationPrompt(details = "which movie had song Dus Bahane 2.0")
recommendations = provider.query(prompt = prompt_question)
recommendations.response

In [22]:
queries = [
    "Can you explain the themes explored in recent Bollywood hits?",
    "What are the main differences between the book and Bollywood movie versions of recent adaptations?",
    "Can you analyze the cinematography style of recent Bollywood blockbusters?",
    "How do recent Bollywood films address contemporary social issues?",
    "What are the key elements that make recent Bollywood movies stand out in terms of storytelling?",
    "Can you discuss the impact of music in recent Bollywood hits on their success?",
    "How do recent Bollywood movies portray cultural diversity and inclusivity?",
    "Can you provide an in-depth character analysis of popular characters in recent Bollywood films?",
    "How do recent Bollywood movies use humor and satire to convey important messages?",
    "Can you explain the significance of recent Bollywood movies in shaping popular culture?",
    "How do recent Bollywood films tackle taboo subjects with sensitivity?",
    "Can you analyze the use of symbolism in recent Bollywood hits?",
    "What are the philosophical themes explored in recent Bollywood movies?",
    "How do recent Bollywood films depict relationships and romance?",
    "Can you discuss the narrative techniques used in recent Bollywood hits?"
]

responses = []

for query in queries:
    prompt_question = RecommendationPrompt(details=query)
    recommendations = provider.query(prompt=prompt_question)
    responses.append(recommendations.response)

# Print responses
for query, response in zip(queries, responses):
    print(f"Query: {query}\nResponse: {response}\n")

Query: Can you explain the themes explored in recent Bollywood hits?
Response: Themes explored in recent Bollywood hits include women empowerment, medical ethics, responsibility, challenges faced by healthcare professionals, corruption, political intrigue, consequences of unethical decisions, and factors contributing to farmers' suicides in India. These themes are depicted through various characters and storylines in the films, shedding light on societal issues and personal struggles.

Query: What are the main differences between the book and Bollywood movie versions of recent adaptations?
Response: The main differences between the book and Bollywood movie versions of recent adaptations include the portrayal of characters, the pacing of the storyline, the inclusion or exclusion of certain plot points, the visual representation of settings and events, and the overall interpretation of the original source material.

Query: Can you analyze the cinematography style of recent Bollywood bloc

In [23]:
new_queries = [
    "What are the latest releases in the Bollywood industry?",
    "Can you list the top-grossing Bollywood movies of 2024 so far?",
    "What are some of the most critically acclaimed Bollywood films released in the past year?",
    "Which Bollywood movies are currently trending on streaming platforms?",
    "What new Bollywood movies have been added to streaming platforms this month?",
    "Can you recommend some new indie Bollywood films that have received positive reviews?",
    "What are the latest releases in the Bollywood horror genre?",
    "Which recently released Bollywood movies are being talked about at film festivals?",
    "What are the newest Bollywood additions to international film festivals?",
    "Can you list some upcoming Bollywood movies that are highly anticipated?",
    "What are the latest Bollywood films by acclaimed directors?",
    "What new animated Bollywood films are set to release this year?",
    "What are the top-rated Bollywood movies on review aggregator websites released in the past six months?",
    "Can you recommend some newly released Bollywood documentaries?",
    "What are some recent Bollywood movies that have been adapted from bestselling books?"
]

responses = []

for query in new_queries:
    prompt_question = RecommendationPrompt(details=query)
    recommendations = provider.query(prompt=prompt_question)
    responses.append(recommendations.response)

# Print responses
for query, response in zip(new_queries, responses):
    print(f"Query: {query}\nResponse: {response}\n")

Query: What are the latest releases in the Bollywood industry?
Response: The latest releases in the Bollywood industry are "Rocky Aur Rani Kii Prem Kahaani" and "Jawan" according to The Hindu, Vogue India, Film Companion, Harper's Bazaar India, and Rediff.

Query: Can you list the top-grossing Bollywood movies of 2024 so far?
Response: I cannot provide information on the top-grossing Bollywood movies of 2024 based on the context provided.

Query: What are some of the most critically acclaimed Bollywood films released in the past year?
Response: Some of the most critically acclaimed Bollywood films released in the past year include "Gangubai Kathiawadi," "Jalsa," "12th Fail," "Jawan," and "Rocky Aur Rani Kii Prem Kahaani."

Query: Which Bollywood movies are currently trending on streaming platforms?
Response: "Dil Bechara," "Soorarai Pottru," "Tanhaji," "Sadak 2," and "Laxmii" are currently among the most searched Bollywood movies on Google in 2020 in India.

Query: What new Bollywood m